In [84]:
# This program predicts stock prices buy using machine learning models

#Install the depedencies

In [85]:
import quandl
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [86]:
#Get the stock data
df = quandl.get("WIKI/AAPL")
# Take a look at the data
print(df.tail())

              Open    High     Low    Close      Volume  Ex-Dividend  \
Date                                                                   
2018-03-21  175.04  175.09  171.26  171.270  35247358.0          0.0   
2018-03-22  170.00  172.68  168.60  168.845  41051076.0          0.0   
2018-03-23  168.39  169.92  164.94  164.940  40248954.0          0.0   
2018-03-26  168.07  173.10  166.44  172.770  36272617.0          0.0   
2018-03-27  173.68  175.15  166.92  168.340  38962839.0          0.0   

            Split Ratio  Adj. Open  Adj. High  Adj. Low  Adj. Close  \
Date                                                                  
2018-03-21          1.0     175.04     175.09    171.26     171.270   
2018-03-22          1.0     170.00     172.68    168.60     168.845   
2018-03-23          1.0     168.39     169.92    164.94     164.940   
2018-03-26          1.0     168.07     173.10    166.44     172.770   
2018-03-27          1.0     173.68     175.15    166.92     168.340  

In [87]:
#Get the Adjusted Close Price
df = df[['Adj. Close']]
print(df.head())

            Adj. Close
Date                  
1980-12-12    0.422706
1980-12-15    0.400652
1980-12-16    0.371246
1980-12-17    0.380362
1980-12-18    0.391536


In [88]:
# A variable for predicting 'n' days out into the future
forecast_out = 30

#Create another column (the target or dependent variable) and shifted '
#n' units up
df['Prediction'] = df[['Adj. Close']].shift(-forecast_out)
#print the new data set
print(df.tail())

            Adj. Close  Prediction
Date                              
2018-03-21     171.270         NaN
2018-03-22     168.845         NaN
2018-03-23     164.940         NaN
2018-03-26     172.770         NaN
2018-03-27     168.340         NaN


In [89]:
### Create the independent data set (X) ###
#Convert the datafraome to a  numpy array
X = np.array(df.drop(['Prediction'],1))
#Remove the last 'n' rows
X = X[:-forecast_out]
print(X)

[[  0.42270592]
 [  0.40065169]
 [  0.37124607]
 ...
 [155.32      ]
 [155.97      ]
 [162.71      ]]


In [90]:
 ### Create the depdendent data set (Y) ###
# Convert the dataframe to a numpy array (All of the values including the
#NaN's)
y = np.array(df['Prediction'])
#Get all of the y values except the last 'n' rows
y = y[:-forecast_out]
print(y)

[  0.47049006   0.45578725   0.43917307 ... 164.94       172.77
 168.34      ]


In [91]:
# Split the data into 80% training and 20% testing
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [92]:
#Create and train the Support Vector Machine (Regressor)
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_rbf.fit(x_train, y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [93]:
# Testing Model: Score returns the coefficent of determination R^2 of the prediction
#The best possible score is 1.0
svm_confidence = svr_rbf.score(x_test, y_test)
print("svm confidence: ", svm_confidence)

svm confidence:  0.9915109231529813


In [94]:
# Create and train the Linear Regression model
lr = LinearRegression()
# Train the model
lr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [95]:
# Testing Model: Score returns the coefficent of determination R^2 of the prediction
#The best possible score is 1.0
lr_confidence = lr.score(x_test, y_test)
print("lr confidence: ", lr_confidence)

lr confidence:  0.9906042741158508


In [96]:
# Set x_forecast equal to the last 30 rows of the original data set from Adj. Close column
x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]
#print(x_forecast)

In [97]:
# Print linear regression model predictions for the next 'n' days
lr_prediction = lr.predict(x_forecast)
print(lr_prediction)

[168.26343156 171.36358501 177.11370462 176.54073897 175.94731025
 175.14925095 176.71467497 179.73809195 183.23215929 182.63873058
 182.36247928 179.17024206 180.40825714 181.03238044 180.8789075
 179.20093665 181.15515879 184.26554378 186.04582992 184.25531225
 182.68988823 182.90475035 182.26016399 179.47718795 179.41579877
 175.35388154 172.87273562 168.87732333 176.88861097 172.35604338]


In [98]:
# Print Support Vector Regressor model predictions for the next 'n' days
svm_prediction = svr_rbf.predict(x_forecast)
print(svm_prediction)

[156.0983335  177.84692653 172.40059996 173.04482855 172.51470702
 169.96158089 172.94252184 174.20627761 176.69623827 176.90682685
 176.82338606 172.51981102 175.82043022 176.49593347 176.39877729
 172.60599758 176.54957857 172.04430103 143.40293815 172.13235276
 176.91667164 176.91693631 176.78858753 173.42306541 173.23739452
 170.7204042  173.73136837 155.87816496 172.74865817 177.12323763]
